In [69]:
!pip install imbalanced-learn


In [ ]:
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import BorderlineSMOTE
import os
import cv2
import numpy as np

In [ ]:
Directory = "dataset/field_and_road"

In [150]:
# Load and preprocess the images
def load_images(directory, label):
    images = []
    for filename in os.listdir(directory):
        img = cv2.imread(os.path.join(directory, filename))
        if img is not None:
            img = cv2.resize(img, (224, 224))  # Resize to MobileNetV2 input size
            images.append(img)
    return np.array(images), np.full(len(images), label)

field_dir = "dataset/field_and_road/field"
road_dir = "dataset/field_and_road/road"

field_images, field_labels = load_images(field_dir, label=1)  # Assign class label 1 for "field"
road_images, road_labels = load_images(road_dir, label=0)     # Assign class label 0 for "road"

# Combine the images and labels
all_images = np.concatenate((field_images, road_images), axis=0)
all_labels = np.concatenate((field_labels, road_labels), axis=0)

# Apply Borderline-SMOTE oversampling to the "field" class
smote = BorderlineSMOTE(sampling_strategy='auto', random_state=42)
field_resampled, field_resampled_labels = smote.fit_resample(all_images.reshape(-1, 224*224*3), all_labels)

# Save the resampled images to a directory
output_dir = "dataset/oversampled_field"
os.makedirs(output_dir, exist_ok=True)

for i, img in enumerate(field_resampled):

    cv2.imwrite(os.path.join(output_dir, f"field_image_{i}.jpg"), img.reshape(224, 224, 3))